##Info
<!-- 

To run this notebook, click menu Cell -> Run All

Workflow:
    1. Sum:
        WW
        GWI
        BSF
        Runoff
        dfs0 inflow
    2. Sum:
        WWTP flow
        MH Spilling
        Outfalls
        Delta volume

 -->

In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *
import subprocess
import sqlite3
from datetime import datetime as dt, timedelta

In [2]:
# sql function
def sql_to_df(sql,model):
  con = sqlite3.connect(model)
  df = pd.read_sql(sql, con)
  con.close()
  return df

In [69]:
flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']
cover_types = ['Normal','Spilling']
# boundary_inflows = []
# boundary_inflows.append('Landfill')

for m in master_list:
  
    model_area = m[0]
    model = m[1]
    result_folder = m[2]
    output_folder = m[3]
    result_list = m[4]
    groupby_acronym_owner = m[5]
    element_filter = m[7]
    wwtp_pipe = m[9]
    outfall_summary = m[10]
    
    outfall_disconnection_df = pd.read_csv(outfall_summary)
    
    #find subfolders for MIKE+
    if model[-7:] == '.sqlite':
        result_dict = {}
        not_founds = []
        for r in result_list:
            file = r[1]
            file_found = False
            for f1 in os.listdir(result_folder):
                if f1[-7:] == '.sqlite':
                    #browse subfolder
                    result_subfolder = os.path.basename(f1)[:-7] + '_m1d - Result Files'
                    try:
                        for f2 in os.listdir(result_folder + '\\' + result_subfolder):
                            if os.path.basename(f2) == file:
                                result_dict[file] = [f1,'\\' + result_subfolder]
                                file_found = True
                    except:
                        pass
            if not file_found:
                not_founds.append(file)
 
    output_subfolder = output_folder + '\\All_Mass_Balances'
    if not os.path.isdir(output_subfolder): os.makedirs(output_subfolder) 

    html_path = output_subfolder + '\\Mass_Balance.html'
    f = open(html_path, "w")
    f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style_mb.css">\n')
    f.write('<script src="..\Maps_And_CSS\script_mb.js"></script>\n')
    f.write('<!DOCTYPE html>\n')
    f.write('<html>\n')
    f.write('<head>\n')
    f.write('<meta charset="utf-8">\n')
    f.write('</head>\n')
    f.write('<body>\n\n')

    f.write('<div class="tab">\n')
    for r in result_list:
        tab = r[0]       
        f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
    f.write('</div>\n')
       
    
    for r in result_list:
        
        table_specs = []
        table_specs.append(['Inflow and Outflow',['Total Inflow','Total Outflow']])
        table_specs.append(['Inflow Breakdown',['Runoff','GWI','Wastewater','DWF','BSF','Boundary']])
        table_specs.append(['Outflow Breakdown',['WWTP','Disconnection','Overflow','Spill']])
        
        header = r[0]
        tab = header
        file = r[1]
        model_path = result_folder + '\\' + result_dict[file][0]
        result_network_path = result_folder + '\\' + result_dict[file][1] + '\\' + file
        result_runoff_path = result_network_path[:-16] + 'Surface_runoff.res1d'
        
        individual_dfs = []
        
        print('Importing ' + header)
        
        f.write('<div id="' + tab + '" class="tabcontent">\n')  
        f.write('<h1>' + tab + '</h1>')
        
        f.write('<div class="row"><div class="column">\n')
       
        print('Import DWF')
        sql = "SELECT ms_DPProfileD.ScheduleID AS Day_Type, ms_DPPatternD.Sqn AS [Hour], Sum(msm_Loadpoint.loadflow*ms_DPPatternD.DPValue) AS Wastewater "
        sql += "FROM ((msm_Loadpoint INNER JOIN msm_BBoundary ON msm_Loadpoint.LoadCategoryNo = msm_BBoundary.LoadCategoryNo) INNER JOIN ms_DPProfileD ON msm_BBoundary.DPProfileID = ms_DPProfileD.ProfileID) INNER JOIN ms_DPPatternD ON ms_DPProfileD.PatternID = ms_DPPatternD.PatternID "
        sql += "WHERE msm_Loadpoint.Active = 1 AND ms_DPProfileD.Active = 1 AND ms_DPPatternD.Active = 1 AND msm_BBoundary.Active = 1 "
        sql += "GROUP BY ms_DPProfileD.ScheduleID, ms_DPPatternD.Time "
        sql += "HAVING (LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekday' Or LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekend') AND ms_DPPatternD.Sqn <> 0 "
        sql += "ORDER BY scheduleid, time"
        diurnal_wws = sql_to_df(sql,model_path) 
        
        sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE description = 'Baseflow' and Active = 1"
        gwi = sql_to_df(sql,model_path).iloc[0,0]
        
        diurnal_wws.Hour = diurnal_wws.Hour - 1
#         diurnal_wws.Wastewater = diurnal_wws.Wastewater
        
        sql = "SELECT COUNT(muid) FROM msm_Loadpoint WHERE description = 'BSF' and Active = 1"
        bsf_count = sql_to_df(sql,model_path).iloc[0,0]
        bsf = 0
        bsf_on = True if bsf_count > 0 else False
        if bsf_on:
            print('Import BSF')
            sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE description = 'BSF' and Active = 1"
            bsf = sql_to_df(sql,model_path).iloc[0,0]
            sql = "SELECT SUM(area) FROM msm_Catchment WHERE Active = 1"
            area = sql_to_df(sql,model_path).iloc[0,0]
            ini_rate = round(bsf * 86400 / area * 10000 * 1000, 0)
            ini_no = ini_rate / 11200 
            
        #Extract results
        print('Import WWTP')
        res1d = Res1D(result_network_path)
        sim_start = res1d.time_index.min()
        start = sim_start + timedelta(days=1)
        end = res1d.time_index.max()
        sim_seconds = (end - sim_start).total_seconds()
        timesteps = len(res1d.time_index)-1
        timestep_seconds = sim_seconds / timesteps
        skip_steps = int(86400 / timestep_seconds)
        
        wwtp_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
        wwtp_df['WWTP'] = list(res1d.query.GetReachEndValues(wwtp_pipe, "Discharge"))[skip_steps:]
                
        print('Import Disconnections')
        disc_df = outfall_disconnection_df[outfall_disconnection_df.Type=='Disconnection']
        first_round = True
        has_disc = False
        for index, row in disc_df.iterrows():
            scenario = row['Scenario']
            if scenario in file:
                muid = row['Structure']
                layer = row['Layer']
                outfall = row['Outfall']
                resid = muid
                if layer.lower() != 'msm_link':
                    resid = layer[4:] + ':' + muid

                disc_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
                ts = list(res1d.query.GetReachEndValues(resid, "Discharge"))[skip_steps:]
                disc_df['Outfall'] = outfall
                disc_df['Disconnection'] = ts
                if first_round == True:
                    disc_df_all = disc_df.copy()
                else:
                    disc_df_all = pd.concat([disc_df_all,disc_df])                                                        
                first_round = False
                has_disc = True
                
        if has_disc:
            disconnection_df = disc_df_all.pivot(columns='Outfall', values='Disconnection')
            individual_dfs.append(['Disconnections',disconnection_df])
            disc_df_all = disc_df_all.groupby(disc_df_all.index).agg({'Disconnection': 'sum'}) 
        
        print('Import outfalls')
        outfall_df = outfall_disconnection_df[outfall_disconnection_df.Type=='Overflow'][['Structure', 'Layer', 'Outfall']]
        outfall_df.sort_values(by=['Outfall','Structure'],inplace=True)
        outfall_df.reset_index(drop=True,inplace=True)
        first_round = True
        has_overflow = False
        
        overflow_by_structure_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
        
        for index, row in outfall_df.iterrows():
            
            muid = row['Structure']
            layer = row['Layer']
            outfall = row['Outfall']
            resid = muid
            if layer.lower() != 'msm_link':
                resid = layer[4:] + ':' + muid

            overflow_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
            ts = list(res1d.query.GetReachEndValues(resid, "Discharge"))[skip_steps:]
            overflow_df['MUID'] = muid
            overflow_df['Outfall'] = outfall
            overflow_df['Overflow'] = ts
            
            overflow_by_structure_df[muid + ' (to ' + outfall + ')'] = ts
            
            if first_round == True:
                overflow_df_all = overflow_df.copy()
            else:
                overflow_df_all = pd.concat([overflow_df_all,overflow_df])                                                        
            first_round = False
            has_overflow = True
                
        if has_overflow:       
            overflow_by_outfall_df = overflow_df_all.groupby([overflow_df_all.index,overflow_df_all.Outfall]).agg({'Overflow': 'sum'}) 
            overflow_by_outfall_df.reset_index(level='Outfall', inplace=True)
            overflow_by_outfall_df = overflow_by_outfall_df.pivot(columns='Outfall', values='Overflow')
            overflow_df_all = overflow_df_all.groupby(overflow_df_all.index).agg({'Overflow': 'sum'})
            individual_dfs.append(['Outfalls',overflow_by_outfall_df])
            individual_dfs.append(['Overflow Structures',overflow_by_structure_df])

        has_boundary = False
        print('Import inflow')
        sql = "SELECT tsconnection, timeseriesname FROM msm_BBoundary WHERE active = 1 AND typeno = 9 "
        sql += "AND variationno = 3"
        df = sql_to_df(sql,model_path)
        first_round = True
        for index, row in df.iterrows():
            has_boundary= True
            rel_path = row['tsconnection']
            timeseriesname = row['timeseriesname']
            dfs0_path = os.path.abspath(os.path.join(result_folder, rel_path))
            res = mikeio.read(dfs0_path)
            inflow_df = res.to_dataframe()
            for i, col in enumerate(inflow_df.columns):
                if col == timeseriesname:
                    col_no = i
            inflow_df = inflow_df[[timeseriesname]]
            inflow_df['Boundary'] = timeseriesname
            inflow_df.rename(columns={timeseriesname:'Inflow'},inplace=True)
            inflow_df = inflow_df[['Boundary','Inflow']]
            
            if '(liter per sec)' in str(res.items[col_no]):
                inflow_df.Inflow = inflow_df.Inflow/1000
                
            if first_round == True:
                inflow_df_all = inflow_df.copy()
            else:
                inflow_df_all = pd.concat([inflow_df_all,inflow_df])                                                        
            first_round = False
        
        if has_boundary:
            inflow_by_boundary_df = inflow_df_all.pivot(columns='Boundary', values='Inflow')
            inflow_by_boundary_df = inflow_by_boundary_df.loc[start:end]
            individual_dfs.append(['Boundaries',inflow_by_boundary_df])
            inflow_df_all = inflow_df_all.groupby(inflow_df_all.index).agg({'Inflow': 'sum'}) 
            inflow_df_all.rename(columns={'Inflow':'Boundary'},inplace=True)   
            
        print('Import spill')
        has_spill = False
        first_round = True
        for node in res1d.data.Nodes:
            muid = node.Id
            for i, flood_type in enumerate(flood_types):
                ts = res1d.query.GetNodeValues(muid,flood_type)
                if ts != None:
                    if max(ts) > 0:
                        spill_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
                        spill_df['Node'] = muid
                        spill_df['Spill'] = list(ts)[skip_steps:]
                        if first_round == True:
                            spill_df_all = spill_df.copy()
                        else:
                            spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                        first_round = False
                        has_spill = True
        
        if has_spill:
            spill_by_node_df = spill_df_all.pivot(columns='Node', values='Spill')
            ordered_cols = [muid for muid in spill_by_node_df.sum().sort_values(ascending=False).index]
            spill_by_node_df = spill_by_node_df[ordered_cols]
            individual_dfs.append(['Spills',spill_by_node_df])
            spill_df_all = spill_df_all.groupby(spill_df_all.index).agg({'Spill': 'sum'})
            
        print('Import runoff')
        has_runoff = False
        first_round = True
        if os.path.exists(result_runoff_path):
            res1d = Res1D(result_runoff_path)
            has_runoff = True
            for i, catchment in enumerate(res1d.data.Catchments):
                ts_id = catchment.Id
                if not ' - RDI' in ts_id and not ' - Kinematic wave (B)' in ts_id:
#                     print('Importing catchment ' + str((i+1)/3) + ' of ' + str(len(res1d.data.Catchments)/3) + ': ' + muid)
                    muid = ts_id
                    ts = res1d.query.GetCatchmentValues(muid,'TotalRunOff')
                    runoff_df = pd.DataFrame(index=res1d.time_index)
                    runoff_df['Node'] = muid
                    runoff_df['Runoff'] = ts
                    if first_round == True:
                        runoff_df_all = runoff_df.copy()
                    else:
                        runoff_df_all = pd.concat([runoff_df_all,runoff_df])                                                        
                    first_round = False
                    
                           
            runoff_df_all = runoff_df_all.groupby(runoff_df_all.index).agg({'Runoff': 'sum'})
            
         
                                
        df_all = wwtp_df.copy()
        if has_overflow:
            df_all = pd.merge(df_all, overflow_df_all, left_index=True, right_index=True, how='left')
        else:
            df_all['Overflow'] = 0
        if has_disc:
            df_all = pd.merge(df_all, disc_df_all, left_index=True, right_index=True, how='left')
        else:
            df_all['Disconnection'] = 0
        if has_spill:
            df_all = pd.merge(df_all, spill_df_all, left_index=True, right_index=True, how='left')
        else:
            df_all['Spill'] = 0
        if has_boundary:
            df_all = pd.merge(df_all, inflow_df_all, left_index=True, right_index=True, how='left')
        else:
            df_all['Boundary'] = 0
        df_all['Total Outflow'] = df_all.WWTP + df_all.Spill + df_all.Overflow + df_all.Disconnection
        if has_runoff:
            df_all = pd.merge(df_all, runoff_df_all, left_index=True, right_index=True, how='left')
        else:
            df_all['Runoff'] = 0
        df_all['DateTime'] = df_all.index
        df_all['Hour'] = df_all.DateTime.dt.hour
        df_all['Weekday'] = df_all['DateTime'].dt.day_name()
        df_all['Day_Type'] = 'Weekdays'
        df_all.loc[df_all['Weekday']=='Saturday','Day_Type']='Weekends'
        df_all.loc[df_all['Weekday']=='Sunday','Day_Type']='Weekends'
        df_all = pd.merge(df_all,diurnal_wws[['Day_Type', 'Hour','Wastewater']],on=['Day_Type', 'Hour'],how='inner')
        df_all.set_index('DateTime',inplace=True)
        df_all.sort_index(inplace=True)
        df_all['Wastewater'] = df_all['Wastewater'].rolling('1h').mean()
        df_all.fillna(method='bfill',inplace=True)
        df_all.drop(columns=['Hour','Weekday','Day_Type'],inplace=True)
        df_all['GWI'] = gwi
        df_all['DWF'] = df_all['GWI'] + df_all['Wastewater']
        df_all['BSF'] = bsf
        df_all['Total Inflow'] = df_all.Boundary + df_all.Runoff + df_all.DWF + df_all.BSF
        
        maxes = df_all.max()
        sums = df_all.sum()
        
        for individual_df in individual_dfs:  
            table_specs.append([individual_df[0],list(individual_df[1].columns)])
            maxes = pd.concat([maxes,individual_df[1].max()])
            sums = pd.concat([sums,individual_df[1].sum()])
        
        for table_spec in table_specs:
            items = table_spec[1]
            f.write('<h2>' + table_spec[0] + '</h2>\n')
            f.write('<table>\n')
            f.write('<tr>\n')
            f.write('<th rowspan="2">Description</th>\n') 
            f.write('<th style="text-align: center">Volume</th>\n') 
            f.write('<th style="text-align: center">Peak Flow</th>\n') 
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<th style="text-align: center">ML</th>\n')
            f.write('<th style="text-align: center">L/s</th>\n') 
            f.write('</tr>\n')
            
            for item in items:
                
                f.write('<tr>\n')
                f.write('<td style="text-align: left">' + item + '</td>\n')
                volume = int(sums[item]*timestep_seconds/1000) if not np.isnan(sums[item]) else 0
                f.write('<td style="text-align: right">'+ str(volume) + '</td>\n')
                flow = int(maxes[item]*1000) if not np.isnan(maxes[item]) else 0
                f.write('<td style="text-align: right">'+ str(flow) + '</td>\n')

                f.write('</tr>\n')
            f.write('</table>\n')
           
        f.write('</div>\n')             
        f.write('<div class="column">\n')
        
        grouped_cols = [list(individual_df[1].columns) for individual_df in individual_dfs]
        cols = [col for sublist in grouped_cols for col in sublist]
        
        all_cols = list(df_all.columns) + cols       

        buttons_ons = []
        buttons_ons.append(['Inflow and Outflow',['Total Inflow','Total Outflow']])
        buttons_ons.append(['Inflows',['Runoff','GWI','Wastewater','DWF','Boundary','BSF','Total Inflow']])
        buttons_ons.append(['Outflows',['WWTP','Disconnection','Overflow','Spill','Total Outflow']])
        
        for individual_df in individual_dfs:
            stophere = 9 if individual_df[0] == 'Spills' else len(individual_df[1].columns)
            button_name = individual_df[0] + ' Top 10' if individual_df[0] == 'Spills' else individual_df[0]
            buttons_ons.append([button_name,list(individual_df[1].columns[:stophere])])

#         falses = [False for i in range(sum([len(individual_df[1].columns) for individual_df in individual_dfs]))]
#         individuals = len(falses)
        for button_ons in buttons_ons:
            on_list = []
            for col in all_cols:
                on_list.append(col in button_ons[1])
            button_ons[1] = on_list
#             button_ons[1] += falses
        buttons_ons


        fig = go.Figure()
        for col in df_all.columns:   

            is_visible = True if col in ['Total Inflow','Total Outflow'] else False
            fig.add_trace(go.Scatter(x=df_all.index, 
                                             y = df_all[col], 
                                             mode='lines',name=col, visible=is_visible))
        for individual_df in individual_dfs:
            stophere = 9 if individual_df[0] == 'Spills' else len(individual_df[1].columns)
            
            for col in individual_df[1].columns[:stophere]:
                fig.add_trace(go.Scatter(x=individual_df[1].index, 
                                             y = individual_df[1][col], 
                                             mode='lines',name=col, visible=False, showlegend=True ))


        fig.update_layout(
            title = 'Model inflows and outflows',
            autosize=False,
            width = 1362,
            height=700,
            margin=dict(
                l=50,
                r=50,
                b=25,
                t=35,
                pad=4
                ),
            yaxis_title='Discharge (cms)',                        

            updatemenus=[
                {
                    'buttons': [
                        {
                            'args': [{'visible': button_on[1]}, {'title': button_on[0]}],
                            'label': button_on[0],
                            'method': 'update'
                        }
                        for button_on in buttons_ons
                    ],
                    'direction': 'left',
                    'pad': {'r': 10, 't': 87},
                    'showactive': True,
                    'type': 'buttons',
                    'x': 0.1,
                    'xanchor': 'left',
                    'y': 0.06,
                    'yanchor': 'top'
                }
                ]                                           
            )
#         fig['layout']['yaxis']['range']=[0:]

        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write('</div>\n')
        f.write('</div>\n')
        f.write('</div>\n')

    f.write('</body>\n')
    f.write('</html>\n')
    f.close()
        
        
print('Done')                
                





Importing Nov 15 2021, 2060 Pop
Import DWF
Import WWTP
Import Disconnections
Import outfalls
Import inflow
Import spill
Import runoff
Done
